<a href="https://colab.research.google.com/github/kapamawi/AI/blob/main/2_9_5__AgencySwarm_agencja_uslugowa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Adaptacja: https://github.com/Shubhamsaboo/awesome-llm-apps/tree/main/ai_agent_tutorials/ai_services_agency

# Setup

In [ ]:
!pip install -qq python-dotenv==1.0.1 agency-swarm==0.4.1 streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.8/115.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.9/423.9 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.7/240.7 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.3/390.3 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79

Ten kod instaluje trzy biblioteki Pythona za pomocą managera pakietów pip:

1. python-dotenv w wersji 1.0.1 - biblioteka do wczytywania zmiennych środowiskowych z pliku .env

2. agency-swarm w wersji 0.4.1 - framework do tworzenia systemów wieloagentowych

3. streamlit - biblioteka do tworzenia interaktywnych aplikacji webowych

Flaga -qq sprawia, że pip wyświetla tylko najważniejsze komunikaty podczas instalacji, pomijając szczegółowe logi i ostrzeżenia.

In [ ]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴
added 22 packages in 6s
⠴
⠴3 packages are looking for funding
⠴  run `npm fund` for details
⠴

Ta komenda instaluje narzędzie `localtunnel` za pomocą menedżera pakietów npm (Node Package Manager).

`localtunnel` to narzędzie, które tworzy publiczny URL dla lokalnie uruchomionego serwera. Pozwala to na udostępnienie aplikacji działającej na lokalnym komputerze przez internet - inni użytkownicy mogą uzyskać do niej dostęp przez wygenerowany adres URL.

Jest to szczególnie przydatne podczas:
- testowania webhooków
- demonstrowania aplikacji klientom/współpracownikom
- debugowania aplikacji w różnych środowiskach
- udostępniania lokalnej aplikacji bez konieczności wdrażania jej na serwer produkcyjny

# App

In [ ]:
from agency_swarm import Agent, Agency, set_openai_key, BaseTool

In [ ]:
help(Agency)

Help on class Agency in module agency_swarm.agency.agency:

class Agency(builtins.object)
 |  Agency(agency_chart: List, shared_instructions: str = '', shared_files: Union[str, List[str]] = None, async_mode: Literal['threading', 'tools_threading'] = None, send_message_tool_class: Type[agency_swarm.tools.send_message.SendMessageBase.SendMessageBase] = <class 'agency_swarm.tools.send_message.SendMessage.SendMessage'>, settings_path: str = './settings.json', settings_callbacks: agency_swarm.agency.agency.SettingsCallbacks = None, threads_callbacks: agency_swarm.agency.agency.ThreadsCallbacks = None, temperature: float = 0.3, top_p: float = 1.0, max_prompt_tokens: int = None, max_completion_tokens: int = None, truncation_strategy: dict = None)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, agency_chart: List, shared_instructions: str = '', shared_files: Union[str, List[str]] = None, async_mode: Literal['threading', 'tools_threading'] = None, send_message_tool_class: Type[agency_swa

In [ ]:
%%writefile muh_app.py
from typing import List, Literal, Dict, Optional
from agency_swarm import Agent, Agency, set_openai_key, BaseTool
from pydantic import Field, BaseModel
import streamlit as st

# narzędzia
class AnalyzeProjectRequirements(BaseTool):
    project_name: str = Field(..., description="Nazwa projektu")
    project_description: str = Field(..., description="Opis projektu i cele")
    project_type: Literal["Aplikacja webowa", "Aplikacja mobilna", "Rozwój API",
                         "Analityka danych", "Rozwiązanie AI/ML", "Inne"] = Field(..., description="Typ projektu")
    budget_range: Literal["10k-25k zł", "25k-50k zł", "50k-100k zł", "100k+ zł"] = Field(...,
                         description="Zakres budżetu projektu")

    class ToolConfig:
        name = "analyze_project"
        description = "Analizuje wymagania i wykonalność projektu"
        one_call_at_a_time = True

    def run(self) -> str:
        """Analizuje projekt i zapisuje wyniki w współdzielonym stanie"""
        if self._shared_state.get("project_analysis", None) is not None:
            raise ValueError("Analiza projektu już istnieje. Proszę przejść do specyfikacji technicznej.")

        analysis = {
            "name": self.project_name,
            "type": self.project_type,
            "complexity": "wysoka",
            "timeline": "6 miesięcy",
            "budget_feasibility": "w zakresie",
            "requirements": ["Skalowalna architektura", "Bezpieczeństwo", "Integracja API"]
        }

        self._shared_state.set("project_analysis", analysis)
        return "Analiza projektu zakończona. Proszę przejść do specyfikacji technicznej."

class CreateTechnicalSpecification(BaseTool):
    architecture_type: Literal["monolityczna", "mikroserwisy", "serverless", "hybrydowa"] = Field( ..., description="Proponowany typ architektury")
    core_technologies: str = Field(...,  description="Lista głównych technologii i frameworków oddzielonych przecinkami" )
    scalability_requirements: Literal["wysoka", "średnia", "niska"] = Field( ...,  description="Wymagania skalowalności" )

    class ToolConfig:
      name = "create_technical_spec"
      description = "Tworzy specyfikację techniczną na podstawie analizy projektu"
      one_call_at_a_time = True

    def run(self) -> str:
        """Tworzy specyfikację techniczną na podstawie analizy"""
        project_analysis = self._shared_state.get("project_analysis", None)
        if project_analysis is None:
            raise ValueError("Proszę najpierw przeanalizować wymagania projektu używając narzędzia AnalyzeProjectRequirements.")

        spec = {
            "project_name": project_analysis["name"],
            "architecture": self.architecture_type,
            "technologies": self.core_technologies.split(","),
            "scalability": self.scalability_requirements
        }

        self._shared_state.set("technical_specification", spec)
        return f"Utworzono specyfikację techniczną dla {project_analysis['name']}."

def init_session_state() -> None:
    """Initialize session state variables"""
    if 'messages' not in st.session_state:
        st.session_state.messages = []
    if 'api_key' not in st.session_state:
        st.session_state.api_key = None

def main() -> None:
    st.set_page_config(page_title="Agencja Usług AI", layout="wide")
    init_session_state()

    st.title("🚀 Agencja Usług AI")

    # Konfiguracja API
    with st.sidebar:
        st.header("🔑 Konfiguracja API")
        openai_api_key = st.text_input(
            "Klucz API OpenAI",
            type="password",
            help="Wprowadź swój klucz API OpenAI, aby kontynuować"
        )

        if openai_api_key:
            st.session_state.api_key = openai_api_key
            st.success("Klucz API zaakceptowany!")
        else:
            st.warning("⚠️ Proszę wprowadzić klucz API OpenAI, aby kontynuować")
            st.markdown("[Uzyskaj swój klucz API tutaj](https://platform.openai.com/api-keys)")
            return

    # Initialize agents with the provided API key
    set_openai_key(st.session_state.api_key)
    api_headers = {"Authorization": f"Bearer {st.session_state.api_key}"}

    # Formularz projektu
    with st.form("project_form"):
        st.subheader("Szczegóły Projektu")

        project_name = st.text_input("Nazwa Projektu")
        project_description = st.text_area(
            "Opis Projektu",
            help="Opisz projekt, jego cele i szczególne wymagania"
        )

        col1, col2 = st.columns(2)
        with col1:
            project_type = st.selectbox(
                "Typ Projektu",
                ["Aplikacja webowa", "Aplikacja mobilna", "Rozwój API",
                 "Analityka danych", "Rozwiązanie AI/ML", "Inne"]
            )
            timeline = st.selectbox(
                "Oczekiwany Termin", ["1-2 miesiące", "3-4 miesiące", "5-6 miesięcy", "6+ miesięcy"]
            )

        with col2:
            budget_range = st.selectbox(
                "Zakres Budżetu",
                ["10k-25k zł", "25k-50k zł", "50k-100k zł", "100k+ zł"]
            )
            priority = st.selectbox( "Priorytet Projektu", ["Wysoki", "Średni", "Niski"] )

        tech_requirements = st.text_area(
            "Wymagania Techniczne (opcjonalnie)", help="Szczególne wymagania techniczne lub preferencje" )

        special_considerations = st.text_area(
            "Specjalne Uwagi (opcjonalnie)",
            help="Dodatkowe informacje lub specjalne wymagania"
        )

        submitted = st.form_submit_button("Analizuj Projekt")

    if submitted and project_name and project_description:
        try:
          # Set OpenAI key
          set_openai_key(st.session_state.api_key)

          # Create agents
          ceo = Agent(
              name="Dyrektor Projektu",
              description="Jesteś CEO wielu firm w przeszłości i masz duże doświadczenie w ocenie projektów i podejmowaniu strategicznych decyzji.",
              instructions="""
              Jesteś doświadczonym CEO, który ocenia projekty. Postępuj ściśle według tych kroków:

              1. NAJPIERW użyj narzędzia AnalyzeProjectRequirements z:
                  - project_name: Nazwa z szczegółów projektu
                  - project_description: Pełny opis projektu
                  - project_type: Typ projektu (Aplikacja webowa, Aplikacja mobilna, itp.)
                  - budget_range: Określony zakres budżetu

              2. POCZEKAJ na zakończenie analizy przed kontynuacją.

              3. Przejrzyj wyniki analizy i przedstaw strategiczne rekomendacje.
              """,
              tools=[AnalyzeProjectRequirements],
              api_headers=api_headers,
              temperature=0.7,
              max_prompt_tokens=25000
          )

          cto = Agent(
              name="Architekt Techniczny",
              description="Starszy architekt techniczny z głęboką wiedzą w projektowaniu systemów.",
              instructions="""
              Jesteś architektem technicznym. Postępuj ściśle według tych kroków:

              1. POCZEKAJ na zakończenie analizy projektu przez CEO.

              2. Użyj narzędzia CreateTechnicalSpecification z:
                  - architecture_type: Wybierz spośród monolityczna/mikroserwisy/serverless/hybrydowa
                  - core_technologies: Lista głównych technologii oddzielonych przecinkami
                  - scalability_requirements: Wybierz wysoka/średnia/niska na podstawie potrzeb projektu

              3. Przejrzyj specyfikację techniczną i przedstaw dodatkowe rekomendacje.
              """,
              tools=[CreateTechnicalSpecification],
              api_headers=api_headers,
              temperature=0.5,
              max_prompt_tokens=25000
          )

          product_manager = Agent(
                              name="Kierownik Produktu",
                              description="Doświadczony kierownik produktu skupiający się na doskonałości dostarczania.",
                              instructions="""
                              - Zarządzaj zakresem i harmonogramem projektu, przedstawiając mapę drogową projektu
                              - Zdefiniuj wymagania produktu i powinieneś zaproponować potencjalne produkty i funkcje, które można zbudować dla startupu
                              """,
                              api_headers=api_headers,
                              temperature=0.4,
                              max_prompt_tokens=25000
                          )

          developer = Agent(
              name="Główny Programista",
              description="Starszy programista z doświadczeniem full-stack.",
              instructions="""
              - Zaplanuj implementację techniczną
              - Przedstaw szacunki nakładu pracy
              - Oceń wykonalność techniczną
              """,
              api_headers=api_headers,
              temperature=0.3,
              max_prompt_tokens=25000
          )

          client_manager = Agent(
              name="Kierownik ds. Sukcesu Klienta",
              description="Doświadczony kierownik ds. klienta skupiający się na dostarczaniu projektu.",
              instructions="""
              - Zapewnij satysfakcję klienta
              - Zarządzaj oczekiwaniami
              - Obsługuj feedback
              """,
              api_headers=api_headers,
              temperature=0.6,
              max_prompt_tokens=25000
          )

          # Utwórz agencję
          agency = Agency(
              [
                  ceo, cto, product_manager, developer, client_manager,
                  [ceo, cto],
                  [ceo, product_manager],
                  [ceo, developer],
                  [ceo, client_manager],
                  [cto, developer],
                  [product_manager, developer],
                  [product_manager, client_manager]
              ],
              async_mode='threading',
              shared_files='shared_files'
          )

          # Przygotuj informacje o projekcie
          project_info = {
              "name": project_name,
              "description": project_description,
              "type": project_type,
              "timeline": timeline,
              "budget": budget_range,
              "priority": priority,
              "technical_requirements": tech_requirements,
              "special_considerations": special_considerations
          }

          st.session_state.messages.append({"role": "user", "content": str(project_info)})
          # Utwórz zakładki i uruchom analizę
          with st.spinner("Agencja Usług AI analizuje Twój projekt..."):
              try:
                  # Pobierz analizę od każdego agenta używając agency.get_completion()
                  ceo_response = agency.get_completion(
                      message=f"""Przeanalizuj ten projekt używając narzędzia AnalyzeProjectRequirements:
                      Nazwa Projektu: {project_name}
                      Opis Projektu: {project_description}
                      Typ Projektu: {project_type}
                      Zakres Budżetu: {budget_range}

                      Użyj dokładnie tych wartości z narzędziem i poczekaj na wyniki analizy.""",
                      recipient_agent=ceo
                  )

                  cto_response = agency.get_completion(
                      message=f"""Przejrzyj analizę projektu i utwórz specyfikacje techniczne używając narzędzia CreateTechnicalSpecification.
                      Wybierz najbardziej odpowiednie:
                      - architecture_type (monolityczna/mikroserwisy/serverless/hybrydowa)
                      - core_technologies (lista oddzielona przecinkami)
                      - scalability_requirements (wysoka/średnia/niska)

                      Oprzyj swoje wybory na wymaganiach projektu i analizie.""",
                      recipient_agent=cto
                  )

                  pm_response = agency.get_completion(
                      message=f"Przeanalizuj aspekty zarządzania projektem: {str(project_info)}",
                      recipient_agent=product_manager,
                      additional_instructions="Skup się na dopasowaniu produktu do rynku i rozwoju mapy drogowej, oraz koordynuj z zespołami technicznymi i marketingowymi."
                  )

                  developer_response = agency.get_completion(
                      message=f"Przeanalizuj implementację techniczną na podstawie specyfikacji CTO: {str(project_info)}",
                      recipient_agent=developer,
                      additional_instructions="Przedstaw szczegóły implementacji technicznej, optymalny stos technologiczny, który byłby używany, włącznie z kosztami usług chmurowych (jeśli są) i opinią o wykonalności, oraz koordynuj z kierownikiem produktu i CTO, aby zbudować wymagane produkty dla startupu."
                  )

                  client_response = agency.get_completion(
                      message=f"Przeanalizuj aspekty sukcesu klienta: {str(project_info)}",
                      recipient_agent=client_manager,
                      additional_instructions="Przedstaw szczegółową strategię wejścia na rynek i plan pozyskiwania klientów, oraz koordynuj z kierownikiem produktu."
                  )

                  # Utwórz zakładki dla różnych analiz
                  tabs = st.tabs([
                      "Analiza Projektu CEO",
                      "Specyfikacja Techniczna CTO",
                      "Plan Kierownika Produktu",
                      "Implementacja Programisty",
                      "Strategia Sukcesu Klienta"
                  ])

                  with tabs[0]:
                      st.markdown("## Analiza Strategiczna CEO")
                      st.markdown(ceo_response)
                      st.session_state.messages.append({"role": "assistant", "content": ceo_response})

                  with tabs[1]:
                      st.markdown("## Specyfikacja Techniczna CTO")
                      st.markdown(cto_response)
                      st.session_state.messages.append({"role": "assistant", "content": cto_response})

                  with tabs[2]:
                      st.markdown("## Plan Kierownika Produktu")
                      st.markdown(pm_response)
                      st.session_state.messages.append({"role": "assistant", "content": pm_response})

                  with tabs[3]:
                      st.markdown("## Plan Rozwoju Głównego Programisty")
                      st.markdown(developer_response)
                      st.session_state.messages.append({"role": "assistant", "content": developer_response})

                  with tabs[4]:
                      st.markdown("## Strategia Sukcesu Klienta")
                      st.markdown(client_response)
                      st.session_state.messages.append({"role": "assistant", "content": client_response})

              except Exception as e:
                  st.error(f"Błąd podczas analizy: {str(e)}")
                  st.error("Sprawdź swoje dane wejściowe i klucz API, a następnie spróbuj ponownie.")

        except Exception as e:
            st.error(f"Błąd podczas analizy: {str(e)}")
            st.error("Sprawdź swój klucz API i spróbuj ponownie.")

    # Dodaj zarządzanie historią w pasku bocznym
    with st.sidebar:
        st.subheader("Opcje")
        if st.checkbox("Pokaż Historię Analiz"):
            for message in st.session_state.messages:
                with st.chat_message(message["role"]):
                    st.markdown(message["content"])

        if st.button("Wyczyść Historię"):
            st.session_state.messages = []
            st.rerun()

if __name__ == "__main__":
    main()

Overwriting muh_app.py


Kod przedstawia aplikację Streamlit służącą do analizy i zarządzania projektami IT przy użyciu agentów AI. Składa się z następujących kluczowych elementów:

1. Klasa `AnalyzeProjectRequirements` - narzędzie do analizy wymagań projektu:
- Przyjmuje nazwę, opis, typ projektu i zakres budżetu
- Zapisuje wyniki analizy w pamięci współdzielonej
- Ocenia złożoność, harmonogram i wykonalność budżetową projektu

2. Klasa `CreateTechnicalSpecification` - narzędzie do tworzenia specyfikacji technicznej:
- Określa typ architektury, główne technologie i wymagania skalowalności
- Działa na podstawie wcześniejszej analizy projektu
- Generuje szczegółową specyfikację techniczną

3. Funkcja `init_session_state()` inicjalizuje zmienne sesji Streamlit do przechowywania wiadomości i klucza API.

4. Główna funkcja `main()`:
- Konfiguruje interfejs użytkownika Streamlit
- Zawiera formularz do wprowadzania szczegółów projektu
- Tworzy i konfiguruje pięciu agentów:
  - Dyrektor Projektu (CEO)
  - Architekt Techniczny (CTO)
  - Kierownik Produktu
  - Główny Programista
  - Kierownik ds. Sukcesu Klienta

5. Logika przetwarzania formularza:
- Sprawdza poprawność wprowadzonych danych
- Tworzy agencję łączącą wszystkich agentów
- Uruchamia analizę projektu przez każdego agenta
- Wyświetla wyniki w zakładkach interfejsu

6. Panel boczny zawiera:
- Pole do wprowadzania klucza API OpenAI
- Opcje wyświetlania i czyszczenia historii analiz

System wykorzystuje architekturę wieloagentową do kompleksowej analizy projektów IT z różnych perspektyw biznesowych i technicznych.

In [ ]:
help(Agency)

NameError: name 'Agency' is not defined

# Biegnij Forrest

In [ ]:
!streamlit run muh_app.py --server.address=localhost  &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.75.128.191
⠙your url is: https://wicked-berries-tap.loca.lt


Ta sekwencja komend uruchamia kilka procesów:

1. `streamlit run muh_app.py --server.address=localhost`
- Uruchamia aplikację Streamlit
- Parametr `--server.address=localhost` określa, że serwer ma być dostępny tylko lokalnie
- `&>/content/logs.txt` przekierowuje wszystkie komunikaty (standardowe wyjście i błędy) do pliku logs.txt
- `&` na końcu uruchamia proces w tle

2. `npx localtunnel --port 8501`
- Tworzy tunel przez localtunnel do portu 8501 (domyślny port Streamlit)
- Pozwala na dostęp do lokalnej aplikacji z internetu
- Wygeneruje publiczny URL dostępowy

3. `curl ipv4.icanhazip.com`
- Pobiera publiczny adres IP komputera
- Może być przydatne do weryfikacji połączenia lub konfiguracji

Znak `&` pomiędzy komendami sprawia, że wszystkie procesy uruchamiają się równolegle.